In [ ]:
from google.colab import drive
drive.mount("/content/drive")
import os
import sys
sys.path.append("/drive/MyDrive/2022_DVA")
path = "drive/MyDrive/2022_DVA/"

In [ ]:
import glob
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)

## Don't DO : Concat All Files
- condat all
- create new column 'date' : format 'yyyyMMdd'
- sort by date desc

In [ ]:
csv_files = [i for i in glob.glob(path+'cleaned/*.csv')]

In [ ]:
all = pd.concat([pd.read_csv(f) for f in csv_files])

In [ ]:
all.info()

In [ ]:
all['date'] = all['tweetcreatedts'].str[:4] + all['tweetcreatedts'].str[5:7] + all['tweetcreatedts'].str[8:10]

In [ ]:
all = all.sort_values(by=['date'])

In [ ]:
all.head()

In [ ]:
all.to_csv(path+'all.csv')

## Starts from Here : Split Hashtags
- Get Top 25 hashtags
- Create new column for each

In [ ]:
df = pd.read_csv(path+'preprocessed/all.csv')

In [ ]:
df['split'] = 0
df['split'] = df['split'].astype('object')

wlist = []
for i in range(len(df)):
    wlist.append(df['hashtags'][i][1:-1].replace("'", "").lower().split(", "))

for i in range(len(df)):
    df.at[i, 'split'] = wlist[i]

In [ ]:
tags = dict()

for words in df.split:
    for word in words:
        if word in tags.keys():
            tags[word] += 1
        else:
            tags[word] = 1

In [ ]:
tags_sort = {k: v for k, v in sorted(tags.items(), key=lambda item: item[1], reverse=True)}

In [ ]:
top30 = list(tags_sort.keys())[:31]
top30 = top30[:2] + top30[3:]

In [ ]:
top30

In [ ]:
df.head()

In [ ]:
for i in range(len(top30)):
    df[top30[i]] = [top30[i] in x for x in df['split']]

In [ ]:
# df.to_csv(path+'preprocessed/splitted.csv')

In [ ]:
df.iloc[0,:]

In [ ]:
df_org = df

## 1. Table for Worldmap
- A map shows the number of twitter which includes TOP 30 tags from 202202 ~ 202210
- group by country
- tooltip will show:
  - hashtag
  - country
  - the number of tweet avg
  - retweet avg
  - retweet max

In [ ]:
def hashtag(df, column):
    df_tmp = df[df[column]]
    df_tmp['hashtag'] = column
    return df_tmp

In [ ]:
df_1 = hashtag(df, top30[0])
df_2 = hashtag(df, top30[1])
df_3 = hashtag(df, top30[2])
df_4 = hashtag(df, top30[3])
df_5 = hashtag(df, top30[4])
df_6 = hashtag(df, top30[5])
df_7 = hashtag(df, top30[6])
df_8 = hashtag(df, top30[7])
df_9 = hashtag(df, top30[8])
df_10 = hashtag(df, top30[9])
df_11 = hashtag(df, top30[10])
df_12 = hashtag(df, top30[11])
df_13 = hashtag(df, top30[12])
df_14 = hashtag(df, top30[13])
df_15 = hashtag(df, top30[14])
df_16 = hashtag(df, top30[15])
df_17 = hashtag(df, top30[16])
df_18 = hashtag(df, top30[17])
df_19 = hashtag(df, top30[18])
df_20 = hashtag(df, top30[19])
df_21 = hashtag(df, top30[20])
df_22 = hashtag(df, top30[21])
df_23 = hashtag(df, top30[22])
df_24 = hashtag(df, top30[23])
df_25 = hashtag(df, top30[24])
df_26 = hashtag(df, top30[25])
df_27 = hashtag(df, top30[26])
df_28 = hashtag(df, top30[27])
df_29 = hashtag(df, top30[28])
df_30 = hashtag(df, top30[29])

In [ ]:
df_c = pd.concat([df_1, df_2, df_3, df_4, df_5, df_6, df_7, df_8, df_9, df_10,
                df_11, df_12, df_13, df_14, df_15, df_16, df_17, df_18, df_19, df_20,
                df_21, df_22, df_23, df_24, df_25, df_26, df_27, df_28, df_29, df_30])

In [ ]:
df_rt = df_c[['location', 'text', 'retweetcount', 'hashtag']]
df_rt = df_rt.rename(columns={'retweetcount':"retweet_max"})
df_rt

In [ ]:
df = df_c
df['count'] = 1

In [ ]:
df = df.groupby(['location', 'hashtag']).agg({
    'count': [np.sum],
    'retweetcount':[np.mean, np.max],
})

In [ ]:
df.to_csv(path+'preprocessed/topHashtag_avg_tweet.csv')

In [ ]:
df = pd.read_csv(path+'preprocessed/topHashtag_avg_tweet.csv')

In [ ]:
df

In [ ]:
df = df[2:]
df = df.rename(columns={'Unnamed: 0':"location", 'Unnamed: 1':"hashtag", 'count':"cnt", 'retweetcount':'retweet_mean', 'retweetcount.1':"retweet_max"})

In [ ]:
df.cnt = df.cnt.astype(int)
df.retweet_mean = df.retweet_mean.astype(float)
df.retweet_max = df.retweet_max.astype(int)

In [ ]:
df['retweet_mean'] = df['retweet_mean'].round(decimals=2)

In [ ]:
df_f = pd.merge(df, df_rt, how="left", on=["location", "hashtag", "retweet_max"])
df_f

In [ ]:
df_f.to_csv(path+'preprocessed/topHashtag_avg_tweet_f.csv')

In [ ]:
df_f = pd.read_csv(path+'preprocessed/topHashtag_avg_tweet_f.csv')

In [ ]:
df_f.drop_duplicates()

In [ ]:
df_f.to_csv(path+'preprocessed/topHashtag_avg_tweet_f.csv')

## 2. Table for Linegraph

## Crate New Table
- groupby date, country
- count top25 hashtags each
- avg retweetcount
- retweet max text

In [ ]:
df = pd.read_csv(path+'preprocessed/split.csv')

In [ ]:
df['count'] = 1

In [ ]:
df.head()

In [ ]:
df_g = df.groupby(["location", "date"]).agg({
    'count': [np.sum],
    'retweetcount':[np.mean, np.max],
    'Ukraine':[np.sum],
    'Russia':[np.sum],
    'StandWithUkraine':[np.sum],
    'Putin':[np.sum],
    'Russian':[np.sum],
    'news':[np.sum],
    'Mariupol':[np.sum],
    'NATO':[np.sum],
    'UkraineRussiaWar':[np.sum],
    'Ukrainian':[np.sum],
    'UkraineWar':[np.sum],
    'USA':[np.sum],
    'Kyiv':[np.sum],
    'RussiaUkraineWar':[np.sum],
    'business':[np.sum],
    'China':[np.sum],
    'StopRussia':[np.sum],
    'War':[np.sum],
    'UkraineRussianWar':[np.sum],
    'SlavaUkraini':[np.sum],
    'Kharkiv':[np.sum],
    'ArmUkraineNow':[np.sum],
    'Kherson':[np.sum],
    'EU':[np.sum],
    'US':[np.sum],
    })

In [ ]:
df_g.head()

In [ ]:
df_g.to_csv(path+'top25Hashtags.csv')

In [ ]:
df_g = pd.read_csv(path+'top25Hashtags.csv')

In [ ]:
df_g.columns

In [ ]:
df_g = df_g.rename(columns={'count':"cnt", 'Unnamed: 0':"location", 'Unnamed: 1':"date", "retweetcount":"retweet_mean", "retweetcount.1":"retweet_max"})

In [ ]:
df_g = df_g.iloc[2:, :].reset_index(drop=True)

In [ ]:
df_g.head()

In [ ]:
df_g.to_csv(path+'top25Hashtags.csv')

In [ ]:
df_g = df_g.rename(columns={'retweet_max':'retweetcount'})

In [ ]:
df_g = df_g.astype({"date":int, "retweetcount":int})

In [ ]:
tweet = df[['location', 'date', 'retweetcount', 'text']]

In [ ]:
df_f = pd.merge(df_g, tweet, how='left', on=['location', 'date', 'retweetcount'])

In [ ]:
df_f.to_csv(path+'top25Hashtags_f.csv')